In [43]:
import pandas as pd
import re
import numpy as np

In [44]:
users_df = pd.read_json("../data/users.jsonl", lines=True)
sessions_df = pd.read_json("../data/sessions.jsonl", lines=True)
products_df = pd.read_json("../data/products.jsonl", lines=True)
print(users_df)

     user_id             name      city                 street
0        102  Monika Forysiak    Poznań      plac Dębowa 11/53
1        103  Kacper Malewicz   Wrocław   aleja Browarna 79/72
2        104    Tomasz Janiuk    Kraków  ulica Cegielniana 318
3        105    Roksana Mućka     Radom        plac Perłowa 48
4        106     Wiktor Jarka  Warszawa            al. Bema 37
..       ...              ...       ...                    ...
195      297    Błażej Pachla    Kraków      ulica Lisia 09/00
196      298     Cezary Jonak    Gdynia    aleja Kołłątaja 110
197      299     Sylwia Karol   Wrocław       al. Podleśna 999
198      300      Bruno Cisoń     Radom   ulica Malinowa 64/08
199      301        Tola Osik  Szczecin    plac Tulipanowa 386

[200 rows x 4 columns]


In [45]:
# Getting sex out of name
for index, row in users_df.iterrows():
    if row["name"].split(" ")[0][-1] == "a":
        users_df.loc[index, "sex"] = "female"
    else:
        users_df.loc[index, "sex"] = "male"

In [46]:
print(users_df)

     user_id             name      city                 street     sex
0        102  Monika Forysiak    Poznań      plac Dębowa 11/53  female
1        103  Kacper Malewicz   Wrocław   aleja Browarna 79/72    male
2        104    Tomasz Janiuk    Kraków  ulica Cegielniana 318    male
3        105    Roksana Mućka     Radom        plac Perłowa 48  female
4        106     Wiktor Jarka  Warszawa            al. Bema 37    male
..       ...              ...       ...                    ...     ...
195      297    Błażej Pachla    Kraków      ulica Lisia 09/00    male
196      298     Cezary Jonak    Gdynia    aleja Kołłątaja 110    male
197      299     Sylwia Karol   Wrocław       al. Podleśna 999  female
198      300      Bruno Cisoń     Radom   ulica Malinowa 64/08    male
199      301        Tola Osik  Szczecin    plac Tulipanowa 386  female

[200 rows x 5 columns]


In [61]:
for index, row in products_df.iterrows():
    color = re.search("'color': '\w+'", str(row["optional_attributes"]))
    if color != None:
        products_df.loc[index, "color"] = color.group(0)[10:-1]
    else:
        products_df.loc[index, "color"] = None

In [81]:
#Do weryfikacji ceny monitorów
for index, row in products_df.iterrows():
    print(f'Nazwa: {row["product_name"]}\nCena: {row["price"]}\n\n')

Nazwa: Monitor LED Philips 193V5LSB2/10 18.5 " 1366 x 768 px TN
Cena: 458.0


Nazwa: Klawiatura mechaniczna Marvo KG962 EN B
Cena: 169.0


Nazwa: Monitor LED iiyama XUB2390HS-B1 23 " 1920 x 1080 px IPS / PLS
Cena: 829.0


Nazwa: Zestaw bezprzewodowy 2w1 klawiatura + mysz Natec
Cena: 45.0


Nazwa: Monitor LED AOC U27P2; 27 " 3840 x 2160 px IPS / PLS
Cena: 629.0


Nazwa: Monitor LED AOC 24B2XDAM 24 " VA
Cena: 689.0


Nazwa: Monitor LED iiyama Inny 22 " 1680 x 1050 px TN
Cena: 449.0


Nazwa: Klawiatura membranowa Logitech G213
Cena: 168.0


Nazwa: Fotel gamingowy Giosedio FBS004
Cena: 419.0


Nazwa: Pendrive Kingston 100 G3 DT100G3/32GB 32 GB
Cena: 19.94


Nazwa: Słuchawki bezprzewodowe nauszne Red Fighter H4
Cena: 255.0


Nazwa: Klawiatura mechaniczna A4Tech A4TKLA45994
Cena: 267.99


Nazwa: Monitor LED iiyama G-Master 24 " 1920 x 1080 px IPS / PLS
Cena: 799.01


Nazwa: Tablet Huawei Matepad T10s 10.1" 2 GB / 32 GB niebieski
Cena: 799.0


Nazwa: Pamięć RAM Samsung DDR3 8 GB 1600
Cena: 13